## 3.1 Data transferred and ingested in butler and access to data

Get the same ras as we did on the summit

In [6]:
from lsst.daf.butler import Butler
from lsst.daf.butler.registry import Registry
import sqlalchemy

repo = '/repo/embargo'
# raw all colleciton contains all images 
collections=['LATISS/raw/all']
butler = Butler(repo, collections="LATISS/raw/all", instrument="LATISS")
cols = butler.registry.queryCollections("*LATISS*raw*")
for c in cols :
    print(c)


butler = Butler(repo, collections=collections)
registry = butler.registry



LATISS/raw/all


In [7]:

dt='raw'
where="exposure.day_obs = 20230511"
datasetRefs = list(registry.queryDatasets(datasetType=dt,where=where))

print(f"There are {len(datasetRefs)} {dt} in collection: {collections} in: {repo} for {where}")

count = 0
for ref in datasetRefs[0], datasetRefs[-1]:
    count = count + 1
    print( ref.dataId['detector'], ref.dataId['exposure'])



There are 592 raw in collection: ['LATISS/raw/all'] in: /repo/embargo for exposure.day_obs = 20230511
0 2023051100001
0 2023051100599


In [8]:
#Make a map of thoose for look up below .. 
dslookup = {}
for ref in datasetRefs:
    dslookup[ref.dataId['exposure']] = ref

### latency ..
"Measure the delays between completion of readout for each image and the ingestion time recorded in the Butler Registry."
The end of readou is the exposure.timespan.end.
Will have to lookup the ingest time usinf a sql query sine is not exposed directly in butler.

In [11]:
from astropy import time

res = butler.registry.queryDimensionRecords('exposure', where=where, instrument="LATISS")
print (f"Got Metadata for {res.count()} exposures")
times = []
min = 100
max = 0
for count,r in enumerate(res):
    dr = dslookup[r.id]
    id = dr.id
    
    if r.id != dr.dataId['exposure']:
        print(f"The sets do not match up {r.id} != {dr.dataId['exposure']}")
        break
    q_string = f"SELECT ingest_date FROM oga.dataset WHERE id='{id}'"
    with registry._db.query(sqlalchemy.text(q_string)) as q:
        intime = time.Time(q.all()[0][0])
        
    diff = (intime - r.timespan.end ).to('minute')
    if diff.value < min:
        min = diff.value
    if diff.value > max:
        max = diff.value 
    # print(r.id,r.timespan.end, intime, diff )
    times.append(diff.value)
    
import numpy as np
print (f"The min time was {min} max time was {max} minutes")
print (f"Time between shutter close and ingest mean:{np.mean(times)}, std:{np.std(times)}, var:{np.var(times)} minutes")

Got Metadata for 592 exposures
The min time was 0.19347863333343263 max time was 0.5866245333332909 minutes
Time between shutter close and ingest mean:0.23482382218469838, std:0.018058551157743862, var:0.0003261112699168522 minutes
